In [1]:
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()
import rosbag_pandas
import pandas as pd
import matplotlib.pyplot as plt
import easygui
import rosbag
import json
import numpy as np
import cPickle as pkl
import time
import seaborn as sns
import datetime as dt
import fnmatch
import os
import bokeh.palettes as bp
from bokeh.layouts import column
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead, Div
from bokeh.io import export_png
from bokeh.plotting import figure, output_file, show, gridplot,save
from datetime import datetime

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)
    
def pickler( obj, path):
    """
    Pickle a Python object
    """
    with open(path, "wb") as pfile:
        pkl.dump(obj, pfile)

def depickler( path):
    """
    Extracts a pickled Python object and returns it
    """
    with open(path, "rb") as pfile:
        data = pkl.load(pfile)
    return data
import os

def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

# Run the above function and store its results in a variable.   
full_file_paths = get_filepaths("/Users/johnny/Desktop/TEST")

In [6]:

def bokehPlot(df, case, parameters, fig=None,
              TOOLS="pan,crosshair,wheel_zoom\
              ,box_zoom,reset,box_select,lasso_select,undo,redo,save",
              x_range=(506, 526), y_range=(506, 526),
              output_backend="webgl", plot_width=500, plot_height=500,
              s=4, xc=513, yc=505,
              xw=20, yw=20, xi=506., yi=503.,
              showPlot=True, title=None, addOdour=True,
              addSmallTit=True, addStart=True, addArrow=True,reallign=True):
    dfc = df[df.trajectory__case == case]
    x = dfc.trajectory__pPos_x
    y = dfc.trajectory__pPos_y
    h = np.deg2rad(dfc.trajectory__pOri_x)

    if 'haw' in parameters['fly']:
        dfcv = dfc.trajectory__valve2
        dfcvP = dfc.trajectory__valve1 #other odour incase of error in gui


    elif 'apple' in parameters['fly']:
        dfcv = dfc.trajectory__valve1
        dfcvP = dfc.trajectory__valve2 #other odour incase of error in gui

    ox = dfc[dfcv == True].trajectory__pPos_x
    oy = dfc[dfcv == True].trajectory__pPos_y
    
    oxP = dfc[dfcvP == True].trajectory__pPos_x
    oyP = dfc[dfcvP == True].trajectory__pPos_y

    
    #     opf=dfc[dfc.trajectory__valve1==True].trajectory__pOri_x

    #     import matplotlib as mpl
    #     colors = [
    #         "#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*mpl.cm.viridis(mpl.colors.Normalize()(opf))
    #     ]


    try:
        cm = bp.viridis(max(parameters['odourQuad']) + 1)
        fc = cm[parameters["odourQuad"][case]]
    except TypeError:
        fc = bp.viridis(10)[5]

    if not addSmallTit:
        title = ''

    if fig is None:
        fig = figure(tools=TOOLS, x_range=x_range, y_range=y_range, output_backend=output_backend,
                     plot_width=plot_width, plot_height=plot_height,
                     active_scroll='wheel_zoom', title=title)

    fig.title.align = 'center'
    fig.title.text_font_size = '14pt'

    r = 2
    xs = parameters['playerInitPos'][0]
    ys = parameters['playerInitPos'][1] + 8
    theta = parameters['windQuadOpen'][case] + 180
    if reallign:
        xoffs = xs - 514
        yoffs = ys - 513
    else:
        xoffs=0
        yoffs=0
    # Pos and heading of fly
    fig.triangle(x - xoffs, y - yoffs, size=s, angle=h, fill_alpha=0.5, line_color=None)

    if addOdour:
        # circle at odour pos, with pf encoded in color
        fig.circle(ox- xoffs, oy- yoffs, size=2 * s, fill_alpha=0.8, line_color=None, fill_color=fc)
        fig.circle(oxP- xoffs, oyP- yoffs, size=2 * s, fill_alpha=0.8, line_color=None, fill_color=(255,0,0))

        
    if addStart:
        # triangle at init pos
        fig.triangle(parameters['playerInitPos'][0], parameters['playerInitPos'][1],
                     size=3 * s, angle=0, fill_alpha=0.9, line_color=None, color='firebrick')

    # arrow

    if parameters["windQuad"][case] == -3:
        lw = 0
        la = 0.1
    elif parameters["windQuad"][case] == -2:
        lw = 4
        la = 1
    else:
        lw = 10
        la = 1
    # print 'le is',lw
    if addArrow:
        fig.add_layout(Arrow(end=VeeHead(size=20), line_color="red", line_alpha=la,

                             x_start=xs, y_start=ys, line_width=lw,
                             x_end=xs + r * np.cos(np.deg2rad(theta)), y_end=ys + r * np.sin(np.deg2rad(theta))))

    return fig

def bokehQuadPlot(df,parameters,path, tr=None,tl=None,bl=None,br=None,
                  TOOLS="pan,crosshair,wheel_zoom,box_zoom,\
                  reset,box_select,lasso_select,undo,redo,save",
                  x_range=(506,526), y_range=(506,526),plot_width=1000, 
                  plot_height=1000,
                 output_backend="webgl",showPlot=False,exposeReturn=False,
                 addBigTit=True,addSmallTit=True, 
                  addArrow=True,addStart=True,addOdour=True):
    
    '''
    plot a single df
    '''

    # output to static HTML file (with CDN resources)
    output_file(path+".html", title="Trajectory Quad", mode="cdn")


    # create a new plot with the tools above, and explicit ranges
    w=500
    h=500 
    s=6
    xc=516
    yc=513
    xw=20
    yw=20
    xi=xc-xw/2.
    yi=yc-yw/2.

    
    
    ls=parameters['loadingString']
    wq=parameters['windQuad']
    oq=parameters['odourQuad']
    wqo=parameters['windQuadOpen']
    
    def quadTitGen(quad,ls=ls,wq=wq,oq=oq,wqo=wqo):
        tit='odour : '+str(oq[quad])+'\t wind : '+str(wq[quad])+'\t windDir : '+str(wqo[quad])
#         tit='ls : '+str(ls)+'\t odour : '+str(oq[quad])+'\t wind : '+str(wq[quad])+'\t windDir : '+str(wqo[quad])

        return tit
    
    
    bl=bokehPlot(df,2,parameters,TOOLS=TOOLS,title=quadTitGen(2),
                 fig=bl,addSmallTit=addSmallTit,addStart=addStart,
                 addArrow=addArrow,addOdour=addOdour
                 ,x_range=x_range, y_range=y_range)
    tr=bokehPlot(df,0,parameters,x_range=bl.x_range, y_range=bl.y_range,TOOLS=TOOLS,title=quadTitGen(0),
                 fig=tr,addSmallTit=addSmallTit,addStart=addStart,
                 addArrow=addArrow,addOdour=addOdour)
    tl=bokehPlot(df,1,parameters,x_range=bl.x_range, y_range=bl.y_range,TOOLS=TOOLS,title=quadTitGen(1),
                 fig=tl,addSmallTit=addSmallTit,addStart=addStart
                 ,addArrow=addArrow,addOdour=addOdour)
    br=bokehPlot(df,3,parameters,x_range=bl.x_range, y_range=bl.y_range,TOOLS=TOOLS,title=quadTitGen(3),
                 fig=br,addSmallTit=addSmallTit,addStart=addStart,
                 addArrow=addArrow,addOdour=addOdour)


    p = gridplot([[tl, tr], [bl, br]])
#     export_png(p, filename=path+".png")
    if addBigTit:
        p=column(Div(text=path.split('/')[-1]),p)
    
    if exposeReturn:#if you want access to all plots
        return p,tr,tl,bl,br
    else:
            
        if showPlot:
            show(p)
        else:
            save(p)

        return p,None,None,None,None


def pickle2df(path=None):
    '''Load a dataframe from pickle'''
    defaultPath="/home/rhagoletis/catkin/src/World/bags/"
    if path is None:
        path=easygui.fileopenbox(title="Bags to Dataframes"
                                  ,default=defaultPath,
                                  multiple=False,filetypes=["*df.pickle"])
        
#     print path, "\n"
#     for path in paths:
#     print "\n\ncurrentl;y analysisnh",path
    # picklepath=path+"_df.pickle"
    data=depickler(path)
    df=data["df"]
    parameters=data['metadata']['parameters']

    return df,parameters,path




def pickle2bokeh(path=None,showPlot=True,exposeReturn=False,tr=None,tl=None,bl=None,br=None,
                 addBigTit=True,addSmallTit=False,addArrow=False,addStart=False,addOdour=False,
                 x_range=(506,526), y_range=(506,526)):
    
    df,parameters,path=pickle2df(path=path)
    try:
        plot,tr,tl,bl,br=bokehQuadPlot(df=df,parameters=parameters,path=path,showPlot=showPlot,
                                       exposeReturn=exposeReturn,tr=tr,tl=tl,bl=bl,br=br,
                                       addBigTit=addBigTit,addSmallTit=addSmallTit, addArrow=addArrow,
                                       addStart=addStart,addOdour=addOdour,
                                       x_range=x_range, y_range=y_range)
    except SyntaxError as e:
        print "\n\n\n Exception",e
        return None,None,None,None
    
    if exposeReturn:#if you want access to all plots
        return plot,df,parameters,path,tr,tl,bl,br
    else:
        return plot,df,parameters,path


def fileList(foldPath,included=['*bag_df.pickle'],excluded=[]):

    
    matches = []
    for root, dirnames, filenames in os.walk(foldPath):
        
        for include in included:
            for filename in fnmatch.filter(filenames, include):
                toKeep=True
                
                for exclude in excluded:
                    if exclude in filename:
                        toKeep=False
                    
                if toKeep:
                    matches.append(os.path.join(root, filename))
    matches.sort()
    return matches


def pickleDir2bokeh(foldPath=None,showPlot=False,
                    addBigTit=True,addSmallTit=True,addArrow=True,
                    addStart=True,addOdour=False,
                    x_range=(506, 526), y_range=(506, 526),
                   defaultPath='/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/agg/odour+wind/',):
    tstart = datetime.now()

    if foldPath is None:
        foldPath= easygui.diropenbox(    default=defaultPath)
    
    matches=fileList(foldPath,included=['*bag_df.pickle'],excluded=['pf'])


    matches.sort(reverse=True)
    print matches
    
    i=1 #file progress index
    for path in matches:
        
        tic()
        plot, df,parameters,path =pickle2bokeh(path,showPlot=showPlot,
                          addBigTit=addBigTit,addSmallTit=addSmallTit,
                           addArrow=addArrow,addStart=addStart,
                          x_range=x_range, y_range=y_range,addOdour=addOdour)

#         df,parameters=pickle2df(path)
        
#         try:
#             bokehQuadPlot(df=df,parameters=parameters,path=path)
#         except Exception as e:
#             print "\n\n\n Exception",e
#             pass
        
        print('\n\n Currently anal on {}. \n It is {} / {} file'.format(path.split('/')[-1], i,len(matches)))

        i+=1
        toc()
    
    tend = datetime.now()
    print "Total time",tend - tstart

    

# def dfDir2bokeh():
    
#     from datetime import datetime

#     tstart = datetime.now()



#     foldPath= easygui.diropenbox(    default="/home/rhagoletis/catkin/src/World/bags/")
#     matches = []
#     for root, dirnames, filenames in os.walk(foldPath):
#         for filename in fnmatch.filter(filenames, '*traj.bag_df.pickle'):
#             if 'pf' in filename:
#                 continue
#             matches.append(os.path.join(root, filename))
#     matches.sort(reverse=True)
#     print matches
    
#     i=1
#     for path in matches:
        
#         tic()
        
# #         df,parameters=pickle2bokeh(path)
#         df,parameters,path2=pickle2df(path)
        
#         try:
#             bokehQuadPlot(df=df,parameters=parameters,path=path,showPlot=False)
#         except Exception as e:
#             print "\n\n\n Exception",e
#             pass
        
#         print('\n\n Currently anal on {}. \n It is {} / {} file'.format(path.split('/')[-1], i,len(matches)))

# #         print "Currently analysing", path
#         i+=1
#         toc()
    
#     tend = datetime.now()
#     print "Total time",tend - tstart


def bag2pickle(paths=None,defaultPath="/home/rhagoletis/catkin/src/World/bags/"):
    '''
    Load  bag filesto make into respective dataframes
    '''
    # defaultPath="/home/rhagoletis/catkin/src/World/bags/"
    if paths is None:
        paths=easygui.fileopenbox(title="Bags to Dataframes"
                                  ,default=defaultPath,
                                  multiple=True,filetypes=["*traj.bag"])
    
    print paths, "\n"
    metadata=None
    tic()
    i=1
    for path in paths:
        tic()
        print "starting analysis of file %s , %s / %s files" %(path.split('/')[-1],i,len(paths))
        try:
            df=rosbag_pandas.bag_to_dataframe(path,include=['/trajectory'])
        except Exception as e:
            print "Bag has some nonetype error",path , e 
            df=None
            parameters=None
            picklepath = None
            continue
        bag=rosbag.Bag(path)

        try:
            for topic,msg,t in bag.read_messages(topics='/metadata'):
                a=msg
    #         parameters=json.loads(a.data)
    #         metadata={"meta":parameters}

            metadata=json.loads(a.data)
            parameters=metadata['parameters']

        except :
            print "no such file!, trying the pickle"
            try:
                metadata=depickler(paths[0].split('.bag')[0])
            except IOError:
                metadata=depickler(paths[0].split('.bag')[0]+'.pickle')

            parameters=metadata['parameters']


        obj=dict(df=df,metadata=metadata)
        
        picklepath=path+"_df.pickle"
        pickler(obj,picklepath)
    #     df.to_pickle(picklepath)

        i+=1
        toc()
    print "\nanalysis of %s files complete" %len(paths)
    return df, parameters,picklepath
    
def bagDir2pickle(foldPath=None,defaultPath="/home/rhagoletis/catkin/src/World/bags/",postProcFunc=None,
                  included=['*.bag'],excluded=[]):
    tstart = datetime.now()
    
    if foldPath is None:
        foldPath= easygui.diropenbox(default=defaultPath)
    
    matches=fileList(foldPath,included=included,excluded=excluded)
#     matches = []
#     for root, dirnames, filenames in os.walk(foldPath):
#         for filename in fnmatch.filter(filenames, '*traj.bag_df.pickle'):
#             if 'pf' in filename:
#                 continue
#             matches.append(os.path.join(root, filename))

    matches.sort(reverse=True)
    print matches
    
    i=1 #file progress index
    for path in matches:
        
        try:
            tic()
            df, parameters,picklepath=bag2pickle(paths=[path])
            if df is None:
                continue

            if postProcFunc is not None:
                postProcFunc(df,parameters,picklepath)

            print('\n\n Currently anal on {}. \n It is {} / {} file'.format(path.split('/')[-1], i,len(matches)))

            i+=1
            toc()
        except Exception as e:
            print "something odd in ",path, e

    tend = datetime.now()
    print "Total time",tend - tstart

def bagDir2bokeh(foldPath=None,defaultPath="/home/rhagoletis/catkin/src/World/bags/",showPlot=False,excluded=['full']):
    bagDir2pickle(foldPath,defaultPath=defaultPath,postProcFunc=bokehQuadPlot,excluded=excluded)
# def xDir2y(x2y,included=['*.bag'],excluded=[],defaultPath="/home/rhagoletis/catkin/src/World/bags/"):
    
#     tstart = datetime.now()

#     foldPath= easygui.diropenbox(default=defaultPath)
    
#     matches=fileList(foldPath,included=included,excluded=excluded)
# #     matches = []
# #     for root, dirnames, filenames in os.walk(foldPath):
# #         for filename in fnmatch.filter(filenames, '*traj.bag_df.pickle'):
# #             if 'pf' in filename:
# #                 continue
# #             matches.append(os.path.join(root, filename))

#     matches.sort(reverse=True)
#     print matches
    
#     i=1 #file progress index
#     for path in matches:
        
#         tic()
#         x2y(paths=[path])
        
#         print('\n\n Currently anal on {}. \n It is {} / {} file'.format(path.split('/')[-1], i,len(matches)))

#         i+=1
#         toc()
    
#     tend = datetime.now()
#     print "Total time",tend - tstart

# bag2pickle()
# bagDir2pickle()
# pickle2bokeh()
# pickle2df()
# dfDir2bokeh()


In [3]:
def plotAggreg(foldPath=None,
               defaultPath='/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/agg/odour+wind/',
              x_range=(493,533), y_range=(485,525),):
    if foldPath is None:
        foldPath=easygui.diropenbox(default=defaultPath)
    files=fileList(foldPath,included=['*.bag_df.pickle'])
    tr=None
    tl=None
    bl=None
    br=None
    iTot=len(files)
    i=1
    for item in files:
        plot,df,parameters,path,tr,tl,bl,br=pickle2bokeh(item,exposeReturn=True,tr=tr,tl=tl,bl=bl,br=br,
                     addBigTit=False,addSmallTit=False,addArrow=False,addStart=False,
                        x_range=x_range, y_range=y_range,)
        print ("currently analysing {}, {}/{} files ").format(item.strip('/')[-1],i,iTot)
        i+=1
    show(plot)
    return plot

In [7]:
pickleDir2bokeh(addSmallTit=True,addOdour=True,addArrow=True,addStart=True)

['/home/rhagoletis/catkin/src/World/bags/2018_01_18/2018-01-18__20~27~15_apple70_00_wind_nullSky_nullGrass_gain~8_speed~1.0_bout~15_DC~-0.033_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2018_01_18/2018-01-18__20~14~38_apple70_00_wind_nullSky_nullGrass_gain~8_speed~1.0_bout~15_DC~-0.033_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2018_01_18/2018-01-18__20~09~06_apple70_00_wind_nullSky_nullGrass_gain~8_speed~1.0_bout~15_DC~-0.033_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2018_01_18/2018-01-18__20~01~45_apple70_00_wind_nullSky_nullGrass_gain~8_speed~1.0_bout~15_DC~0.0_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2018_01_18/2018-01-18__19~55~09_apple70_00_wind_nullSky_nullGrass_gain~8_speed~1.0_bout~15_DC~0.0_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/bags/2018_01_18/2018-01-18__19~48~43_apple70_00_wind_nullSky_nullGrass_gain~8_speed~1.0_bout~15_DC~0.0_traj.bag_df.pickle', '/home/rhagoletis/catkin/src/World/b

In [ ]:
pickle2bokeh(path='/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/2017_09/2017_09_28/2017-09-28__22~14~39_apple61_11_odour_wind_nullSky_gain~8_speed~1.0_bout~30_DC~0.0_traj.bag_df.pickle',
             addOdour=False,addArrow=False,addStart=False,addSmallTit=False)

In [10]:
bagDir2bokeh()

['/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/2018/2018_01/2018_01_17/2018-01-17__20~13~09_apple69_00_wind_nullSky_gain~8_speed~1.0_bout~15_DC~-0.219_traj.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/2018/2018_01/2018_01_17/2018-01-17__20~04~03_apple69_00_wind_nullSky_gain~8_speed~1.0_bout~15_DC~-0.219_traj.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/2018/2018_01/2018_01_17/2018-01-17__19~54~53_apple69_00_wind_nullSky_gain~8_speed~1.0_bout~15_DC~-0.219_traj.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/2018/2018_01/2018_01_17/2018-01-17__19~47~21_apple69_00_wind_nullSky_gain~8_speed~1.0_bout~15_DC~-0.219_traj.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/2018/2018_01/2018_01_17/2018-01-17__19~40~05_apple69_00_wind_nullSky_gain~8_speed~1.0_bout~15_DC~-0.219_traj.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/2018/2018_01/2018_01_17/2018-01-17__19~33~19_apple69_00_wind_nullSky_gain~8_speed~

In [ ]:
df=Out[56]
df=df[0]

In [ ]:
xc=514
yc=513
w=15
xw=w
yw=w
xr=(xc-xw,xc+xw)
yr=(yc-yw,yc+yw)

# pickleDir2bokeh(addBigTit=False,addSmallTit=False,addArrow=False,
#                 addStart=False,x_range=xr,y_range=yr,
#                 defaultPath='/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/agg/odour+wind/')
out=plotAggreg(x_range=xr,y_range=yr)

In [ ]:
a=pickle2df('/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_traj.bag_df.pickle')
df=a[0]

In [ ]:
df['run']=((df.trajectory__runNum-1)*4+df.trajectory__trial)

In [ ]:
df3=df[df.run==4]

In [ ]:
fps=165
preE=2
postE=2
preE*=fps
postE*=fps


In [ ]:
df3=df[df.run==4*i]
offI=np.where(df3.trajectory__valve1.diff()==-1)
# offI[0]=np.append(offI[0],len(df3)-1)
offI=np.append(offI[0],len(df3)-1)

offInterval=np.diff(offI)
offInterval=np.append(offInterval,0)
print offI,len(offI)
print offInterval,len(offInterval)

xcoords=offI[offInterval>preE]
xcoordsI=df3.index[xcoords]


for xc in xcoords:
    sx=df3.trajectory__pPos_x[xc]
    sy=df3.trajectory__pPos_y[xc]
    sh=df3.trajectory__pOri_x[xc]
    
    x=df3.trajectory__pPos_x-sx
    y=df3.trajectory__pPos_y-sy
    
    xp=x*np.cos(np.deg2rad(sh))-y*np.sin(np.deg2rad(sh))
    yp=x*np.sin(np.deg2rad(sh))+y*np.cos(np.deg2rad(sh))

    
    plt.scatter(x=xp[xc-preE:xc],
                y=yp[xc-preE:xc],alpha=0.4,s=0.5)
    plt.scatter(x=xp[xc:xc+postE],
                y=yp[xc:xc+postE],alpha=1,s=2)

#     plt.scatter(x=df3.trajectory__pPos_x[xc-preE:xc]-sx,
#                 y=df3.trajectory__pPos_y[xc-preE:xc]-sy,alpha=0.4,s=0.5)
#     plt.scatter(x=df3.trajectory__pPos_x[xc:xc+postE]-sx,
#                 y=df3.trajectory__pPos_y[xc:xc+postE]-sy,alpha=1,s=2)

#     df3.trajectory__pPos_x[xc-preE:xc+postE].plot()
# for xc in xcoordsI:
#     plt.axvline(x=xc)

    
# df3.trajectory__valve1.plot()
plt.show()

In [ ]:
i=1
print i
df3=df[df.run==4*i]
offI=np.where(df3.trajectory__valve1.diff()==-1)
# offI[0]=np.append(offI[0],len(df3)-1)
offI=np.append(offI[0],len(df3)-1)

offInterval=np.diff(offI)
offInterval=np.append(offInterval,0)
print offI,len(offI)
print offInterval,len(offInterval)

xcoords=offI[offInterval>preE]
xcoordsI=df3.index[xcoords]


for xc in xcoords:
    sx=df3.trajectory__pPos_x[xc]
    sy=df3.trajectory__pPos_y[xc]
    sh=-(df3.trajectory__pOri_x[xc])
    print sh
    
    x=df3.trajectory__pPos_x-sx
    y=df3.trajectory__pPos_y-sy
    h=df3.trajectory__pOri_x

#     xp=x
#     yp=y
#     plt.scatter(x=xp[xc-preE:xc],
#                 y=yp[xc-preE:xc],alpha=0.4,s=.5)
#     plt.scatter(x=xp[xc:xc+postE],
#                 y=yp[xc:xc+postE],alpha=1,s=.10)
    
    xp=x*np.cos(np.deg2rad(sh))-y*np.sin(np.deg2rad(sh))
    yp=x*np.sin(np.deg2rad(sh))+y*np.cos(np.deg2rad(sh))

    print 
    plt.scatter(x=xp[xc-preE:xc],
                y=yp[xc-preE:xc],alpha=0.4,s=2)
    plt.scatter(x=xp[xc:xc+postE],
                y=yp[xc:xc+postE],alpha=1,s=10)

plt.axes().set_aspect('equal', 'datalim')
plt.show()

In [ ]:
fps=165
preT=2
preE=3
postE=3
preE*=fps
postE*=fps
preT*=fps

for i in df.run.unique():
    print i
    df3=df[df.run==4*i]
    offI=np.where(df3.trajectory__valve1.diff()==-1)
    # offI[0]=np.append(offI[0],len(df3)-1)
    offI=np.append(offI[0],len(df3)-1)

    offInterval=np.diff(offI)
    offInterval=np.append(offInterval,0)
    print offI,len(offI)
    print offInterval,len(offInterval)

    xcoords=offI[offInterval>preT]
    xcoordsI=df3.index[xcoords]


    for xc in xcoords:
        sx=df3.trajectory__pPos_x[xc]
        sy=df3.trajectory__pPos_y[xc]
        sh=-(df3.trajectory__pOri_x[xc])
        sh=0
        print sh

        x=df3.trajectory__pPos_x-sx
        y=df3.trajectory__pPos_y-sy
        h=df3.trajectory__pOri_x

    #     xp=x
    #     yp=y
    #     plt.scatter(x=xp[xc-preE:xc],
    #                 y=yp[xc-preE:xc],alpha=0.4,s=.5)
    #     plt.scatter(x=xp[xc:xc+postE],
    #                 y=yp[xc:xc+postE],alpha=1,s=.10)

        xp=x*np.cos(np.deg2rad(sh))-y*np.sin(np.deg2rad(sh))
        yp=x*np.sin(np.deg2rad(sh))+y*np.cos(np.deg2rad(sh))

        print 
        plt.scatter(x=xp[xc-preE:xc],
                    y=yp[xc-preE:xc],alpha=0.2,s=10,c='r')
        plt.scatter(x=xp[xc:xc+postE],
                    y=yp[xc:xc+postE],alpha=0.2,s=10,c='g')

plt.axes().set_aspect('equal', 'datalim')
plt.show()

In [ ]:
plt.scatter(x=df3.trajectory__pPos_x,y=df3.trajectory__pPos_y)
plt.show()

In [ ]:
df3.index[offI[offInterval>preE]]

In [ ]:
xcoords=df3.index[offI[offInterval>preE]]

In [ ]:
offI[0][offInterval>preE]

In [ ]:
df3.trajectory__pPos_x.plot()


In [ ]:
df3.trajectory__pPos_x[xcoords[0]:]


In [ ]:
np.where(df3.trajectory__valve1.diff()==-1)`
df3.trajectory__valve1.plot()
plt.show()

In [ ]:
df3.trajectory__valve1.plot()
plt.show()

NameError: name 'df' is not defined

In [12]:

import scipy.interpolate as inter
import numpy as np
import pylab as plt

x = np.array([13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1])
y = np.array([2.404070, 1.588134, 1.760112, 1.771360, 1.860087,
          1.955789, 1.910408, 1.655911, 1.778952, 2.624719,
          1.698099, 3.022607, 3.303135])
xx = np.arange(1,13.01,0.1)
s1 = inter.InterpolatedUnivariateSpline (x, y)
s1rev = inter.InterpolatedUnivariateSpline (x[::-1], y[::-1])
# Use a smallish value for s
s2 = inter.UnivariateSpline (x[::-1], y[::-1], s=0.1)
s2crazy = inter.UnivariateSpline (x[::-1], y[::-1], s=5e8)
plt.plot (x, y, 'bo', label='Data')
# plt.plot (xx, s1(xx), 'k-', label='Spline, wrong order')
plt.plot (xx, s1rev(xx), 'k--', label='Spline, correct order')
# plt.plot (xx, s2(xx), 'r-', label='Spline, fit')
# Uncomment to get the poor fit.
#plt.plot (xx, s2crazy(xx), 'r--', label='Spline, fit, s=5e8')
plt.minorticks_on()
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [14]:
a=pickle2df()
df=a[0]

In [17]:
x=df.trajectory__pPos_x.values
y=df.trajectory__pPos_y.values


array([ 513.        ,  513.        ,  513.        , ...,  519.09887695,
        519.10461426,  519.11035156])

In [29]:
l=2000
x_=x[:l]
y_=y[:l]

In [30]:
plt.plot(x_,y_)
plt.show()

In [ ]:
df2=df.copy()

In [ ]:
a

In [ ]:
df.

In [ ]:
parameters=a[1]

In [ ]:

class PlotData():
    def __init__(self,df,parameters, fig=None,
              x_range=None, y_range=None,
              plot_width=1000, plot_height=1000,
              xc=513, yc=505, xw=20, yw=20,
              showPlot=True, title=None, addOdour=True,
              addStart=True, addArrow=True,reallign=False,xir=513,yir=513,
                 plotFrom=None,plotTo=None,
                 runNumFrom=None,runNumTo=None,
                 caseFrom=None,caseTo=None,filterGui=False

                 ):

        self.df=df
        # self.case=case
        self.parameters=parameters
        self.fig=fig
        self.plot_width=plot_width
        self.plot_height=plot_height
        self.x_range=self.getRange(x_range, xc, xw)
        self.y_range=self.getRange(y_range, yc, yw)

        self.showPlot=showPlot
        self.title=title

        self.df2,self.plotFrom,self.plotTo=self.filterData(
                                            self.df,runNumFrom,runNumTo,
                                            caseFrom,caseTo,
                                            plotFrom,plotTo,
                                            reallign,xir,yir,
                                            filterGui)

        return self.df2

    def filterData(self,df, runNumFrom,runNumTo,
                    caseFrom,caseTo,plotFrom,plotTo,
                    reallign,xir,yir,
                   filterGui=True):

        if filterGui:
            msg = "Enter the start stop of your data"
            title = "Choose runNum/case or explicit index"
            fieldNames = ["runNumFrom", "runNumTo", "caseFrom", "caseTo", "IndexFrom", "IndexTo"]
            fieldValues = multenterbox(msg, title, fieldNames)

            runNumFrom=fieldValues[0]
            runNumTo=fieldValues[1]
            caseFrom=fieldValues[2]
            caseTo=fieldValues[3]
            plotFrom=fieldValues[4]
            plotTo=fieldValues[5]


        if plotFrom is 'start':
            plotFrom=0
        elif (filterGui and plotFrom=='') or (not filterGui and plotFrom is None):
            plotFrom = np.where((df["trajectory__runNum"] == runNumFrom) &
                              (df["trajectory__case"] == caseFrom))[0][0]


        if plotTo is 'end':
            plotTo = df.shape[0]
        elif (filterGui and plotTo=='') or (not filterGui and plotTo is None):
            plotTo = np.where((df["trajectory__runNum"] == runNumTo) &
                              (df["trajectory__case"] == caseTo))[0][0]




        if reallign:
            xs = self.parameters['playerInitPos'][0]
            ys = self.parameters['playerInitPos'][1]

            xoffs = xs - xir
            yoffs = ys - yir
            self.parameters['playerInitPos'][0]-=xoffs
            self.parameters['playerInitPos'][1]-=yoffs


            df.trajectory__pPos_x-=xoffs
            df.trajectory__pPos_y-=yoffs

        return df.ix[plotFrom:plotTo,:],plotFrom,plotTo

    def getRange(self, range, c, w):
        '''
        give the range if range is none as center +- width
        :param range: a tuple containing range, None if need to defined by center and width
        :param c: Center of the range
        :param w: Total width of the range 
        :return: range tuple centered around c, with width w
        '''
        if range is None:
            range=(c-w/2.,c+w/2.)

        return range



class BokehPlot(PlotData):
    def __init__(self,case,TOOLS="pan,crosshair,wheel_zoom\
                ,box_zoom,reset,box_select,lasso_select,undo,redo,save",
                output_backend="webgl",addSmallTit=True,titFontSize='14pt',
                addTraj=True,addOdour=True,addStart=True,addArrow=True,
                trajS=4,odourS=8,startS=10,arrowS=20,
                trajAlpha=0.5,odourAlpha=0.8,startAlpha=0.9,arrowAlpha=0.8,
                trajC='blue',odourC='green',startC='red',arrowC='black'):

        self.df=super(PlotData, self).__init__()

        dfc = self.df[self.df.trajectory__case == case]
        x = dfc.trajectory__pPos_x
        y = dfc.trajectory__pPos_y
        h = np.deg2rad(dfc.trajectory__pOri_x)

        if 'haw' in self.parameters['fly']:
            dfcv = dfc.trajectory__valve2
        elif 'apple' in self.parameters['fly']:
            dfcv = dfc.trajectory__valve1

        ox = dfc[dfcv == True].trajectory__pPos_x
        oy = dfc[dfcv == True].trajectory__pPos_y


        try:
            cm = bp.viridis(max(self.parameters['odourQuad']) + 1)
            fc = cm[self.parameters["odourQuad"][case]]
        except TypeError:
            fc = bp.viridis(10)[5]

        if not addSmallTit:
            title = ''

        if self.fig is None:
            self.fig = figure(tools=TOOLS, x_range=self.x_range, y_range=self.y_range,
                         plot_width=self.plot_width, plot_height=self.plot_height,
                         active_scroll='wheel_zoom', title=title,
                         output_backend=output_backend,)

        self.fig.title.align = 'center'
        self.fig.title.text_font_size = titFontSize

        r = 2
        xs = self.parameters['playerInitPos'][0]
        ys = self.parameters['playerInitPos'][1] 
        # if reallign:
        #     xoffs = xs - 513
        #     yoffs = ys - 513
        # else:
        #     xoffs = 0
        #     yoffs = 0

        # Pos and heading of fly
        if addTraj:
            self.fig.triangle(x, y, size=trajS, angle=h, 
                              fill_alpha=trajAlpha, line_color=None,fill_color=trajC)

        if addOdour:
            # circle at odour pos, with pf encoded in color
            self.fig.circle(ox, oy, size=odourS, fill_alpha=odourAlpha, line_color=None, 
                            fill_color=odourC)

        if addStart:
            # triangle at init pos
            self.fig.triangle(xs,ys,
                         size=startS, angle=self.parameters['playerInitH'], 
                              fill_alpha=startAlpha, line_color=None, color=startC)

        # arrow

        if self.parameters["windQuad"][case] == -3:
            lw = 0
            la = 0.1
        elif self.parameters["windQuad"][case] == -2:
            lw = 4
            la = 1
        else:
            lw = 10
            la = 1
        # print 'le is',lw
        theta = self.parameters['windQuadOpen'][case] + 180

        if addArrow:
            self.fig.add_layout(Arrow(end=VeeHead(size=arrowS), line_color=arrowC, 
                                    line_alpha=la,line_width=lw,
                                    x_start=xs, y_start=ys, 
                                    x_end=xs + r * np.cos(np.deg2rad(theta)), 
                                    y_end=ys + r * np.sin(np.deg2rad(theta))))

        return self.fig
    


In [ ]:
BokehPlot(df=df,parameters=parameters,case=1)

In [ ]:
df=a[0]
reset=np.where(df["trajectory__reset"] == True)[0]
print reset

In [ ]:
import arrow

In [ ]:
import easygui
import arrow

bagPath='/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_full.bag'
runNumStart=12
caseStart=2

# bagPath=easygui.fileopenbox(filetypes=['*.bag'])
# runNumStart=easygui.integerbox(msg='wnter the start runNum')
# caseStart=easygui.integerbox(msg='enter the start case')

# runNumStop=easygui.integerbox(msg='wnter the stop runNum')
# caseStop=easygui.integerbox(msg='enter the start case')

pklP='/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_full.bag_df.pickle'
df,parameters,path=pickle2df(path=pklP)


In [ ]:
df.trajectory__DCoffset[100:103]

In [ ]:
plt.scatter(x=df.trajectory__pPos_x[232743:],y=df.trajectory__pPos_y[232743:])
plt.show()

In [ ]:
reset=np.where(df["trajectory__reset"] == True)[0]
reset

In [ ]:
startI=np.where((df["trajectory__runNum"]==runNumStart)&(df["trajectory__case"]==caseStart))[0][0]
print startI
dt=df.index[startI]

In [ ]:
df.ix[2:,:]

In [ ]:
save(bokehPlot(df,3,parameters,title=''),path+'_3TrajOver.html')
save(bokehPlot(df,3,parameters,title='',addOdour=False),path+'_3Traj.html')
save(bokehPlot(df.ix[startI:,:],3,parameters,title=''),path+'_3SelTrajOver.html')
save(bokehPlot(df.ix[startI:,:],3,parameters,title='',addOdour=False),path+'_3SelTraj.html')

# startI

In [ ]:
from easygui import multenterbox
msg = "Enter the start stop of your data"
title = "Choose runNum/case or explicit index"
fieldNames = ["runNumFrom","runNumTo","caseFrom","caseTo","IndexFrom","IndexTo"]
fieldValues = []  # we start with blanks for the values
fieldValues = multenterbox(msg,title, fieldNames)

# ma

In [ ]:
from easygui import multenterbox
msg = "Enter your personal information"
title = "Credit Card Application"
fieldNames = ["Name","Street Address","City","State","ZipCode"]
fieldValues = []  # we start with blanks for the values
fieldValues = multenterbox(msg,title, fieldNames)

# make sure that none of the fields was left blank
while 1:
    if fieldValues == None: break
    errmsg = ""
    for i in range(len(fieldNames)):
      if fieldValues[i].strip() == "":
        errmsg = errmsg + ('"%s" is a required field.\n\n' % fieldNames[i])
    if errmsg == "": break # no problems found
    fieldValues = multenterbox(errmsg, title, fieldNames, fieldValues)
print "Reply was:", fieldValues

In [ ]:
import easygui
import arrow
# import bokehPlot
import numpy as np
import subprocess

bagPath='/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_traj.bag'
runNumStart=12
caseStart=2

# bagPath=easygui.fileopenbox(filetypes=['*.bag'])
# runNumStart=easygui.integerbox(msg='wnter the start runNum')
# caseStart=easygui.integerbox(msg='enter the start case')
outBag='out.bag'
# runNumStop=easygui.integerbox(msg='wnter the stop runNum')
# caseStop=easygui.integerbox(msg='enter the start case')


df,parameters,path=bag2pickle([bagPath])
startI=np.where((df["trajectory__runNum"]==runNumStart)&(df["trajectory__case"]==caseStart))[0][0]
print startI
dt=df.index[startI]
a=arrow.get(dt)
ts=a.timestamp
ts=ts+a.microsecond


filt = subprocess.Popen(['rosbag','filter',bagPath,outBag,'"t.to_sec()>='+str(ts)+'"'], stdout=subprocess.PIPE)

In [ ]:
bagPath='/home/rhagoletis/catkin/src/World/bags/2017-09-03__17~46~56_apple55_00_odour_wind_gain~16_speed~1.0_bout~30_DC~0.0_traj.bag'
runNumStart=12
caseStart=3
dt=df.index[startI]
a=arrow.get(dt)
ts=a.timestamp
ts=ts+(a.microsecond*.00001)


filt = subprocess.Popen(['rosbag','filter',bagPath,outBag,'"t.to_sec()>='+str(ts)+'"'], stdout=subprocess.PIPE)

In [ ]:
filt.communicate()

In [ ]:
a=arrow.get(dt)
a.timestamp

In [ ]:
arrow.get(arrow.get(dt).timestamp)

In [ ]:
dt2=arrow.get(dt).timestamp
print dt2
arrow.get(dt2)

In [ ]:
arrow.utcnow()

In [ ]:
a[1]["bagFileName"]

In [ ]:
df.index[0]

In [ ]:
arrow.get(df.index[0])

In [ ]:
arrow.get(df.index[0]).to('+05:30')



In [ ]:
(arrow.get(df.index[0]).to('+05:30')).timestamp

In [ ]:
(arrow.get(df.index[0])).timestamp

In [ ]:
dt=df.index[232743]

In [ ]:
df=Out[115][0]

In [ ]:
5>2>1

In [ ]:
((df["trajectory__pPos_x"]>520)).plot()
plt.show()

In [ ]:
((df["trajectory__pPos_x"] > 513.2) & (df["trajectory__pPos_x"] < 513.4) & (df["trajectory__pPos_y"] > 525.8) & (df["trajectory__pPos_y"] < 526))
# plt.show()

In [ ]:
len(df)

In [ ]:
a=np.where(((513.2<df["trajectory__pPos_x"])&(513.4>df["trajectory__pPos_x"])&(525.8<df["trajectory__pPos_y"])&(526>df["trajectory__pPos_y"])))[0]

In [ ]:
list(np.where(df["trajectory__reset"] == True))

In [ ]:
reset=np.where(df["trajectory__reset"] == True)[0]
print reset
pos=241429
reset[np.argmin((pos-reset)*((pos-reset)>0)*1)-1]

In [ ]:
(pos-reset)*((pos-reset)>0)*1

In [ ]:
dfnow = df.ix[200, :]

In [ ]:
from PyKDE4.kdeui import *

In [ ]:
dfnow

In [ ]:
a

In [ ]:
plt.plot(a)
plt.show()

In [ ]:
plt.plot((np.where(((513.05<df["trajectory__pPos_x"])<513.1))))
plt.show()

In [ ]:
a

In [ ]:
# np.where(df["trajectory__reset"] == True)
list(np.where(df["trajectory__reset"] == True)[0])

In [ ]:
df.ix[df.index[df.trajectory__reset==True].tolist()[0]]
# df.index[df['BoolCol']].tolist()

In [ ]:
pickleDir2bokeh(addBigTit=False,addSmallTit=False,addArrow=False,addStart=False)

In [ ]:
bagDir2bokeh(defaultPath='/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31')

In [ ]:
from bokeh.io import export_svgs
plot=Out[21]
plot.output_backend = "svg"
export_svgs(plot, filename="plot.svg")

In [ ]:
dfDir2bokeh()

In [ ]:
easygui.fileopenbox()

In [ ]:
bagDir2pickle(defaultPath=)

In [ ]:
p,df,parameters,path=pickle2bokeh()

In [ ]:
show(bokehPlot(df,2,parameters,plot_width=1280,plot_height=1280))

In [ ]:


plot,df,parameters,path=pickle2bokeh()
# bokehQuadPlot(df,parameters,path)


# dfDir2bokeh()


In [ ]:
showPlot(bokehPlot(df,2,parameters,plot_width=1280,plot_height=1280))